In [25]:
import numpy as np
import random

from collections import namedtuple, deque
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal, MultivariateNormal

import torch.optim as optim
import time
from torch.utils.tensorboard import SummaryWriter
import argparse


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

In [26]:
class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, threshold_vector_size, seed, hidden_size=32, init_w=3e-3, log_std_min=-20, log_std_max=2):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each action
            seed (int): Random seed
            hidden_size (int): Number of nodes in hidden layers
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)     
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, threshold_vector_size)


    def forward(self, state):
        x = F.relu(self.fc1(state), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)
        out = self.fc3(x)
        return out

    
    def get_threshold_vector(self, state):
        """
        returns the action based on a squashed gaussian policy. That means the samples are obtained according to:
        a(s,e)= tanh(mu(s)+sigma(s)+e)
        """
        #state = torch.FloatTensor(state).to(device) #.unsqzeeze(0)
        #action = torch.clamp(action*action_high, action_low, action_high)
        
        threshold_vector = self.forward(state)
        return threshold_vector

In [27]:
class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, threshold_vector_size, number_of_attributes, seed, hidden_size=32):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            threshold_vector_size (int): Dimension of each threshold vector
            seed (int): Random seed
            hidden_size (int): Number of nodes in the network layers

        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size+threshold_vector_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, number_of_attributes)

    def forward(self, state, threshold_vector):
        """Build a critic (value) network that maps (state, threshold_vector) pairs -> Q-values."""
        x = torch.cat((state, threshold_vector), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def get_attribute(self, threshold_vector):
        index_selected_attribute = np.argmax(self.forward(threshold_vector)) # check argmax output
        return index_selected_attribute

In [ ]:
class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            threshold_vector_size (int): dimension of each threshold vector
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.threshold_vector_size = threshold_vector_size
        self.number_of_attributes = number_of_attributes
        self.seed = random.seed(random_seed)
        
        print("Using: ", device)
        
        # actor Network 
        self.actor_local = Actor(state_size, threshold_vector_size, random_seed, hidden_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)     
        
        # critic Network  
        self.critic = Critic(state_size, threshold_vector_size, number_of_attributes, random_seed, hidden_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=LR_CRITIC, weight_decay=0)

        # Replay memory
        self.memory = ReplayBuffer(threshold_vector_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
        

    def step(self, state, action, reward, next_state, done, step):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(step, experiences, GAMMA)
            
    
    def act(self, state):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        
        # add greedy epsilon

        threshold_vector = self.actor_local.get_threshold_vector(state)
        index_selected_attribute = self.critic.get_attribute(threshold_vector)
        
        action = (index_selected_attribute, threshold_vector[index_selected_attribute])

        # update tree

        return action

    def learn(self, step, experiences, gamma):
        """Updates actor, critics using given batch of experience tuples.
        Critic_loss = 
        Actor_loss = 
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        

        # ---------------------------- update critic ---------------------------- #
        # Compute loss
        critic_loss = 
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute loss
        actor_loss = 

        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()


In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
    

In [ ]:
def RLCI(n_episodes=200, max_t=500, print_every=10):
    scores_deque = deque(maxlen=100)
    average_100_scores = []

    for i_episode in range(1, n_episodes+1):

        state = env.reset()
        state = state.reshape((1,state_size))

        avg_score = 0
        for t in range(max_t):

            action = agent.act(state)
            
            next_state, reward, done, info = env.step(action)
            next_state = next_state.reshape((1,state_size))

            agent.step(state, action, reward, next_state, done, t)
            state = next_state
            avg_score += reward

            if done:
                break

        avg_score /= max_t

        scores_deque.append(avg_score)
        writer.add_scalar("Reward", avg_score, i_episode)
        
        print('\rEpisode {} Score: {:.2f}'.format(i_episode, avg_score, end="")
        if i_episode % print_every == 0:
            print('\rEpisode {} Score: {:.2f}'.format(i_episode, avg_score, end="")            
            
    torch.save(agent.actor_local.state_dict(), args.info + ".pt")
    

In [ ]:
seed = 42
n_episodes = 100
GAMMA = 0.99
TAU = 1e-2
HIDDEN_SIZE = 256
BUFFER_SIZE = int(1e6)
BATCH_SIZE = 256


t0 = time.time()
writer = SummaryWriter("runs/")
torch.manual_seed(seed)
np.random.seed(seed)
state_size = 
threshold_vector_size = 
number_of_attributes = 

#env = gym.make(env_name)

agent = Agent(state_size=state_size, threshold_vector_size=threshold_vector_size, number_of_attributes=number_of_attributes, random_seed=seed,hidden_size=HIDDEN_SIZE, action_prior="uniform") #"normal"

RLCI(n_episodes=n_episodes, max_t=500, print_every=10)

t1 = time.time()
print("training took {} min!".format((t1-t0)/60))


Using:  cpu
Episode 1 Reward: -1736.27  Average100 Score: -1736.27

KeyboardInterrupt: ignored